本周只有一个代码实践题

### <评阅点>     
1. 完成代码并无bug  50‘   
2. 代码有部分bug   40'   
3. 代码有重大bug   30‘
4. 代码不完整  20'   
5. 其余 0‘   

以下代码课用来加载 预先训练好的模型,你需要只需要修改模型的存放路径即可（第二行代码）

In [2]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from torch.utils.data import DataLoader,Dataset

第一步 使用以下链接下载相应预训练模型

https://convaisharables.blob.core.windows.net/lsp/multiref/small_ft.pkl

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
weights = torch.load('../lecture13/small_ft.pkl')
medium_config = GPT2Config(n_embd = 768,n_layer = 12, n_head = 12)
model = GPT2LMHeadModel(medium_config)

weights['lm_head.weight'] = weights['lm_head.decoder.weight']
weights.pop('lm_head.decoder.weight',None)

model.load_state_dict(weights)
model.train()

你需要写一个推理函数，这个函数接收一个英文句子为输入，输出一个回应。

### 测试语句 
一轮对话   
1. Does money buy happiness ?   
2. What is the best way to buy happiness?   

一轮对话   
1. what is the meaning of a godd life ?   
2. How to be a good person ?   

直接用Assignment13跑过的模型

In [11]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
medium_config = GPT2Config(n_embd = 768,n_layer = 12, n_head = 12)
model = GPT2LMHeadModel(medium_config)

In [14]:
model.load_state_dict(torch.load('../lecture13/dials_model.pth')['model'])

<All keys matched successfully>

In [15]:
class Dials:
    
    def __init__(self, model):
        self.dials = []
        self.model = model
    
    def get_next_word(self, inputs):
        tokens_tensor = torch.tensor([tokenizer.encode(inputs)])
    
        with torch.no_grad():
            outputs = self.model(tokens_tensor)
            predictions = outputs[0]
        
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        predicted_word = tokenizer.decode([predicted_index])
    
        return predicted_word
    
    def generate_answer(self, inputs, max_num_sentences):
        answer = ''
        num = 0
        while True:
            next_word = self.get_next_word(inputs)
            if next_word == '<|endoftext|>':
                if answer:
                    break
                else:
                    continue
            else:
                answer += next_word
                inputs += next_word
                if answer[-1] in ['.', '?', '!'] and num > 2:
                    break
                else:
                    num += 1
        return answer.strip()
    
    def start_dial(self, max_num_sentences=1):
        print("Hi, I'm Robot Alex.")
        print("Before the chat begins, you need to know following rules:")
        print("If you want to quit the chat, please input: [Quit]")
        print("If you want to clear the chat, please input: [Clear]")
        print("If you want to recall your last input, please input: [Back]")
        print("Let's chat now!")
        
        while True:
            question = input('Q: ').strip()
            if not question:
                print('Please enter something!')
            if question == '[Quit]':
                self.dials = []
                print('Chat end.')
                break
            elif question == '[Clear]':
                print('Chat history is gone.')
                print()
                self.dials = []
                continue
            elif question == '[Back]':
                if len(self.dials) > 0:
                    self.dials.pop(-1)
                    continue
                else:
                    print("Back to where? We don't have any chat history!")
                    continue
            inputs = ' '.join([d+' <|endoftext|> ' for dial in self.dials for d in dial] + 
                              [question + ' <|endoftext|> '])
            answer = self.generate_answer(inputs, max_num_sentences)
            self.dials.append([question, answer])
            print('A: {}'.format(answer))
    

In [58]:
dials = Dials(model)

In [59]:
dials.start_dial()

Hi, I'm Robot Alex.
Before the chat begins, you need to know following rules:
If you want to quit the chat, please input: [Quit]
If you want to clear the chat, please input: [Clear]
If you want to recall your last input, please input: [Back]
Let's chat now!
Q: Hi, I'm back!
A: OK.Thanks for asking me.
Q: I'm going to improve your performance!
A: Oh,,!
Q: Great! Isn't it?!
A: I'll be right.
Q: See you then.
A: Good-bye.
Q: [Quit]
Chat end.


-------------

[top_k&top_p](https://zhuanlan.zhihu.com/p/80211911)实现

In [44]:
# 单独实现
def top_k(distribution, k):
    return torch.topk(distribution, k, sorted=Ture)

def top_p(sorted_values, p):
    cum_prob = torch.cumsum(F.softmax(sorted_values, dim=-1), dim=-1)
    for i in range(len(cum_prob)):
        if cum_prob[i] >= p:
            return i

In [53]:
# top_k top_p 整合
def top_k_top_p_filter(distribution, k, p):
    values, indices = torch.topk(distribution, k, sorted=True)
    
    cum_prob = torch.cumsum(F.softmax(values, dim=-1), dim=-1)
    
    for i in range(len(cum_prob)):
        if cum_prob[i] >= p:
            return values[:i+1], indices[:i+1]

In [99]:
# 编入到生成语句中
def generate(input_encode, score, k, p):
    tokens_tensor = torch.tensor([input_encode])
    with torch.no_grad():
        outputs = model(tokens_tensor)
        distribution = outputs[0][0, -1, :]
        
        values, indices = torch.topk(distribution, k, sorted=True)
        prob = F.softmax(values, dim=-1)
        cum_prob = torch.cumsum(prob, dim=-1)
        
        for i in range(len(cum_prob)):
            if cum_prob[i] >= p:
                logprob = torch.log(prob)
                scored_sentences = [(input_encode+[indices[j].item()], score+logprob[j].item()) for j in range(i+1)]
                return scored_sentences

In [77]:
generate(tokenizer.encode("I'm going to"), 0, 30, 0.9)

[([40, 1101, 1016, 284, 262], 21.79204750061035),
 ([40, 1101, 1016, 284, 467], 21.452220916748047),
 ([40, 1101, 1016, 284, 1011], 21.314449310302734),
 ([40, 1101, 1016, 284, 307], 20.67784309387207),
 ([40, 1101, 1016, 284, 616], 20.27678680419922),
 ([40, 1101, 1016, 284, 326], 20.246797561645508),
 ([40, 1101, 1016, 284, 1281], 20.23945426940918),
 ([40, 1101, 1016, 284, 1309], 19.87432098388672),
 ([40, 1101, 1016, 284, 910], 19.784082412719727),
 ([40, 1101, 1016, 284, 2822], 19.41615104675293),
 ([40, 1101, 1016, 284, 423], 19.239120483398438),
 ([40, 1101, 1016, 284, 869], 18.920625686645508),
 ([40, 1101, 1016, 284, 731], 18.86473846435547),
 ([40, 1101, 1016, 284, 257], 18.7321834564209),
 ([40, 1101, 1016, 284, 2051], 18.695947647094727)]

beam search实现

In [147]:
def beam_search(beam_size, sentence, generate, topk, topp, alpha=1):
    beam = [(tokenizer.encode(sentence), 0)]
    best_sentences = []
    punish = lambda x: ((5+x)**alpha)/((5+1)**alpha)  # x=length of sentence
    sp = len(beam[0][0]) # start point of new sentence
    while beam_size > 0:
        candidates = []
        for sentence_code, score in beam:
            candidates += generate(sentence_code, score, topk, topp)
        beam_candi = sorted(candidates, key=lambda x: x[1]/punish(len(x[0][sp:])), reverse=True)[:beam_size]
        beam = []
        for sentence_code, score in beam_candi:
            if sentence_code[-1] == 50256: # tokenizer.encode('<|endoftext|>'):
                best_sentences.append((sentence_code[sp:-1], score/punish(len(sentence_code[sp:]))))
                beam_size -= 1
            else:
                beam.append((sentence_code, score))
    return best_sentences

In [152]:
best_sentences = beam_search(5, "Does money buy happiness? <|endoftext|>", generate, 20, 0.9)
for sentence_code, score in best_sentences:
    print(tokenizer.decode(sentence_code), score)

Yes. -2.400179802440107
You can buy anything. -2.0109613740986045
You can buy anything online. -1.467979036475299
You can buy it for yourself. -1.6939414302603557
You can buy it for free. -2.3183807042928843


随机取一个回复

In [131]:
import random

In [160]:
def get_a_random_sentence(sentences_and_scores):
    r = random.random()
    scores = [score for _, score in sentences_and_scores]
    prob = F.softmax(torch.tensor(scores), dim=-1)
    cum_prob = torch.cumsum(prob, dim=-1)
    for i in range(len(cum_prob)):
        if r < cum_prob[i]:
            print(cum_prob)
            print(r)
            return sentences_and_scores[i][0]

In [163]:
tokenizer.decode(get_a_random_sentence(best_sentences))

tensor([0.1230, 0.3046, 0.6172, 0.8665, 1.0000])
0.5348381351002666


'You can buy anything online.'

整合到Dial类中

In [164]:
import time

In [174]:
class Dials_v1:
    
    def __init__(self, model, beam_size=5, alpha=1, k=40, p=0.9):
        self.dials = []
        self.model = model
        self.beam_size = beam_size
        self.alpha = alpha
        self.k = k
        self.p = p
        
    
    def generate(self, input_encode, score):
        tokens_tensor = torch.tensor([input_encode])
        with torch.no_grad():
            outputs = model(tokens_tensor)
            distribution = outputs[0][0, -1, :]
        
            values, indices = torch.topk(distribution, self.k, sorted=True)
            prob = F.softmax(values, dim=-1)
            cum_prob = torch.cumsum(prob, dim=-1)
        
            for i in range(len(cum_prob)):
                if cum_prob[i] >= self.p:
                    logprob = torch.log(prob)
                    scored_sentences = [(input_encode+[indices[j].item()], score+logprob[j].item()) for j in range(i+1)]
                    return scored_sentences
                
    def beam_search(self, beam_size, inputs, alpha):
        beam = [(tokenizer.encode(inputs), 0)]
        best_sentences = []
        punish = lambda x: ((5+x)**alpha)/((5+1)**alpha)  # x=length of sentence
        sp = len(beam[0][0]) # start point of new sentence
        while beam_size > 0:
            candidates = []
            for sentence_code, score in beam:
                candidates += self.generate(sentence_code, score)
            beam_candi = sorted(candidates, key=lambda x: x[1]/punish(len(x[0][sp:])), reverse=True)[:beam_size]
            beam = []
            for sentence_code, score in beam_candi:
                if sentence_code[-1] == 50256: # tokenizer.encode('<|endoftext|>'):
                    best_sentences.append((sentence_code[sp:-1], score/punish(len(sentence_code[sp:]))))
                    beam_size -= 1
                else:
                    beam.append((sentence_code, score))
        
        r = random.random()
        scores = [score for _, score in best_sentences]
        prob = F.softmax(torch.tensor(scores), dim=-1)
        cum_prob = torch.cumsum(prob, dim=-1)
        for i in range(len(cum_prob)):
            if r < cum_prob[i]:
                return tokenizer.decode(best_sentences[i][0])
    
    def start_dial(self, showtime=True):
        print("Hi, I'm Robot Alex.")
        print("Before the chat begins, you need to know following rules:")
        print("If you want to quit the chat, please input: [Quit]")
        print("If you want to clear the chat, please input: [Clear]")
        print("If you want to recall your last input, please input: [Back]")
        print("Let's chat now!")
        
        while True:
            question = input('Q: ').strip()
            if not question:
                print('Please enter something!')
            if question == '[Quit]':
                self.dials = []
                print('Chat end.')
                break
            elif question == '[Clear]':
                print('Chat history is gone.')
                print()
                self.dials = []
                continue
            elif question == '[Back]':
                if len(self.dials) > 0:
                    self.dials.pop(-1)
                    continue
                else:
                    print("Back to where? We don't have any chat history!")
                    continue
            
            inputs = ' '.join([d+' <|endoftext|> ' for dial in self.dials for d in dial] + 
                              [question + ' <|endoftext|> '])
            
            start = time.time()
            answer = self.beam_search(self.beam_size, inputs, self.alpha)
            end = time.time()
            time_cost = round(end - start, 1)
            
            self.dials.append([question, answer])
            if showtime:
                print('A: {}[{}s]'.format(answer, time_cost))
            else:
                print('A: {}'.format(answer))
    

In [175]:
dial = Dials_v1(model)

In [176]:
dial.start_dial()

Hi, I'm Robot Alex.
Before the chat begins, you need to know following rules:
If you want to quit the chat, please input: [Quit]
If you want to clear the chat, please input: [Clear]
If you want to recall your last input, please input: [Back]
Let's chat now!
Q: Hello!
A: Hello?[1.7s]
Q: I feel really good today!
A: Me, too. But it's just a day's work.[4.8s]
Q: What's your job?
A: Not too bad.[4.1s]
Q: A banker?
A: I got this job by a job in the summer.[9.7s]
Q: What do you do in your job?
A: I got it.[10.2s]
Q: [Clear]
Chat history is gone.

Q: I want to work as a coder.
A: That sounds good.[2.8s]
Q: But l have learnt accounting for six years.
A: Well, that ’ s ok.[4.9s]
Q: Thank you for supporting me.
A: It ’ s my pleasure.[6.3s]
Q: [Clear]
Chat history is gone.

Q: What is the fastest way to earn money?
A: You have to pay a fine.[4.3s]
Q: Illegal way?
A: Here, pay is your driver's license.[4.7s]
Q: [Clear]
Chat history is gone.

Q: What is the best way to earn money?
A: You should ask

>服务器上跑语料代码不知道为什么断了2次，结果现在还没跑完，用的是上周的5000个对话的语料，可能效果并没有太大的提高。<br>
而且随着对话次数变多，计算速度也变慢了。<br>
网上有资料说可以通过past来提高计算速度，类似于缓存，等研究好了加上去～

>这次主要添加了topktopp筛选以及随机选取回复，效果感觉不错。而且不需要像上次那样通过判断标点来直接截断长度来防止句子无意义过长。<br>
等提高速度后尝试一下中文对话～